In [66]:
import pandas as pd
import pprint
import ast
import json
import numpy as np
from scipy.stats import entropy
from scipy.special import softmax
import numpy as np
import pandas as pd
from scipy.special import softmax
from scipy.stats import entropy
import ast



In [67]:
from dict import news_labels, yahoo_labels, newsgroups_labels, depression_labels, rating_labels, emotion_labels

In [68]:
dictt =  news_labels
model = 'cohere'
dataset_tag = 'ag_news'

In [69]:

def calculate_metric_with_true_label(df, column_name):
    

    def parse_top_logprobs(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except Exception as e:
                print(f"❌ Nie udało się sparsować: {val[:60]}... ➡ {e}")
                return None
        return val

    df['top_logprobs'] = df['top_logprobs'].apply(parse_top_logprobs)
    
    df['logprobs_cal'] = df['top_logprobs'].apply(
    lambda x: np.array([item['logprob'] for item in x if isinstance(item, dict) and 'logprob' in item])
    if isinstance(x, list) else np.array([])
)



    # Przekształcenie logprobs do prawdopodobieństw przez softmax
    df['probs'] = df['logprobs_cal'].apply(
        lambda logprobs: softmax(logprobs) if len(logprobs) > 0 else np.array([])
    )

    # Entropia rozkładu
    df['entropy'] = df['probs'].apply(
        lambda probs: entropy(probs, base=2) if len(probs) > 0 else np.nan
    )

    # Margin: różnica między top-1 a top-2
    df['margin'] = df['probs'].apply(
        lambda probs: np.sort(probs)[-1] - np.sort(probs)[-2] if len(probs) > 1 else np.nan
    )

    # KL divergence względem idealnego rozkładu (1.0 na poprawnej etykiecie)
    def kl_div(probs, top_logprobs, label):
        if not isinstance(top_logprobs, list) or len(probs) == 0:
            return np.nan

        try:
            # lista tokenów jako stringi
            tokens = [str(item['token']).strip().lower() for item in top_logprobs if isinstance(item, dict) and 'token' in item]
            label_str = str(label).strip().lower()

            if label_str not in tokens:
                return np.nan

            match_idx = tokens.index(label_str)
            ideal_dist = np.zeros_like(probs)
            ideal_dist[match_idx] = 1.0

            return entropy(ideal_dist, probs, base=2)

        except Exception as e:
            print(f"⚠️ Błąd w kl_div: {e}")
            return np.nan


    # df['kl_divergence'] = df.apply(
    #     lambda row: kl_div(row['probs'], row['top_logprobs'], row[column_name]), axis=1
    # )
    
    df['kl_divergence'] = df['probs'].apply(
    lambda probs: np.sum(probs * np.log(probs / (1 / len(probs)))) if len(probs) > 0 else np.nan
        )
    
    # df['top_prob'] = df['probs'].apply(lambda p: np.max(p) if len(p) > 0 else np.nan)
    # df['kl_divergence'] = 2 * (1 - df['top_prob'])



    # Normalizacja (min-max)
    for col in ['entropy', 'margin', 'kl_divergence']:
        norm_col = f'normalized_{col}'
        min_val = df[col].min()
        max_val = df[col].max()
        if pd.notnull(min_val) and pd.notnull(max_val) and max_val > min_val:
            df[norm_col] = (df[col] - min_val) / (max_val - min_val)
        else:
            df[norm_col] = 0.0

    return df




In [70]:

prompt = 'no_samples_numbers'
temp = 0
exp = 1

In [71]:

path = f'../NO_EXAMPLES/results/{prompt}_temp{0}/{model}/{dataset_tag}_{model}_{prompt}_temp{temp}_exp{exp}.csv'
df = pd.read_csv(path)
        # df['top_logprobs_formatted'] = df['top_logprobs'].apply(format_top_logprobs)


In [72]:
df

,text,output,logprobs,top_logprobs,original_label
0,Coke is it: BHP coal to double BHP Billiton pl...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business
1,BIG HIKE IN ENERGY BILLS The company says the ...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business
2,Officials Blame Contractors in Tunnel Leak The...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.5...",Business
3,Before the Bell: Krispy Kreme Down 11 Pct Shar...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business
4,Medical Examiner Finds No Injuries on Thanou-S...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.0...",Sports
...,...,...,...,...,...
995,Indian politician sent to prison Top Hindu nat...,0,"{'content': [{'token': '0', 'bytes': [48], 'lo...","[{'token': '0', 'bytes': [48], 'logprob': -0.0...",World
996,Ravens - Bills: Game Notes COMPOSED: With Lewi...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.0...",Sports
997,Dell touts in-home services Dell offers to ins...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Sci/Tech
998,"Lately, it's been fast times World Series anal...",1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.0...",Sports


In [73]:
df['original_label_map'] = df['original_label'].map(dictt)

In [74]:
df['output'] = df['output'].astype(str)
df['original_label_map'] = df['original_label_map'].astype(str)


In [75]:
df['is_error'] = df['original_label_map'] != df['output']

In [76]:
df_metrics = calculate_metric_with_true_label(df, 'original_label_map')
df_metrics.head(17)


,text,output,logprobs,top_logprobs,original_label,original_label_map,is_error,logprobs_cal,probs,entropy,margin,kl_divergence,normalized_entropy,normalized_margin,normalized_kl_divergence
0,Coke is it: BHP coal to double BHP Billiton pl...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business,2,False,"[-0.003556119976565242, -6.253556251525879, -6...","[0.9964517888610972, 0.001923604224309573, 0.0...",0.038194,0.994528,2.969258,0.018293,0.994694,0.981707
1,BIG HIKE IN ENERGY BILLS The company says the ...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business,2,False,"[-0.009564409963786602, -5.259564399719238, -5...","[0.9904829752455664, 0.005197577689923865, 0.0...",0.090749,0.985285,2.932830,0.044711,0.985449,0.955289
2,Officials Blame Contractors in Tunnel Leak The...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.5...",Business,2,True,"[-0.5680442452430725, -1.5680441856384277, -2....","[0.5666347831743106, 0.2084532998072513, 0.126...",1.642682,0.358181,1.857112,0.824853,0.358241,0.175147
3,Before the Bell: Krispy Kreme Down 11 Pct Shar...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business,2,False,"[-0.0011151769431307912, -7.251115322113037, -...","[0.9988982266548715, 0.0007093918346481757, 0....",0.014076,0.998189,2.985976,0.006168,0.998355,0.993832
4,Medical Examiner Finds No Injuries on Thanou-S...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.0...",Sports,1,False,"[-0.06411883234977722, -2.8141188621520996, -6...","[0.9379121376099231, 0.0599587151703046, 0.001...",0.351468,0.877953,2.752113,0.175772,0.878099,0.824228
5,Merck sets up Vioxx review committee BOSTON (C...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business,2,False,"[-0.03429611772298813, -3.5342960357666016, -5...","[0.9662874026982847, 0.02917935358687067, 0.00...",0.236055,0.937108,2.832112,0.117755,0.937264,0.882245
6,Anheuser-Busch: World's Drinking More Bud NEW...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business,2,False,"[-0.0017170461360365152, -7.2517170906066895, ...","[0.9982942788120264, 0.0007089629978122959, 0....",0.020895,0.997585,2.981249,0.009597,0.997751,0.990403
7,FT gets Collins Stewart claim struck out The F...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business,2,False,"[-0.038578346371650696, -4.038578510284424, -4...","[0.9621577077144218, 0.01762253023995156, 0.01...",0.288484,0.944535,2.795770,0.144111,0.944692,0.855889
8,DVR To The Rescue! News reachs us from America...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.6...",Sci/Tech,3,True,"[-0.610644519329071, -1.3606445789337158, -1.8...","[0.5430378756125247, 0.2565129140215307, 0.155...",1.603887,0.286525,1.884002,0.805351,0.286573,0.194649
9,Cambodia rush to solve succession Cambodia rac...,0,"{'content': [{'token': '0', 'bytes': [48], 'lo...","[{'token': '0', 'bytes': [48], 'logprob': -0.0...",World,0,False,"[-0.0012740122620016336, -7.001274108886719, -...","[0.9987559356537683, 0.0009107474377124805, 0....",0.015580,0.997845,2.984933,0.006925,0.998011,0.993075


In [77]:
corr_matrix = df_metrics[['is_error', 'entropy', 'margin', 'kl_divergence']].corr(method='pearson')
print(corr_matrix['is_error'])


is_error         1.000000
entropy          0.409167
margin          -0.380573
kl_divergence   -0.409167
Name: is_error, dtype: float64


In [78]:
# import pandas as pd
# import os

# # Liczenie korelacji
# corr_matrix = df_metrics[['is_error', 'entropy', 'margin', 'kl_divergence']].corr(method='pearson')
# entropy_corr = corr_matrix.loc['is_error', 'entropy']
# margin_corr = corr_matrix.loc['is_error', 'margin']
# kl_corr = corr_matrix.loc['is_error', 'kl_divergence']

# # Przygotowanie wiersza z wynikami
# result_row = pd.DataFrame([{
#     'dataset': dataset_tag,
#     'model': model,
#     'entropy_corr': entropy_corr,
#     'margin_corr': margin_corr,
# }])

# # Ścieżka do pliku
# csv_path = 'pearson_results.csv'

# # Sprawdzenie czy plik istnieje
# if os.path.exists(csv_path):
#     # Dopisz do istniejącego pliku
#     result_row.to_csv(csv_path, mode='a', header=False, index=False)
# else:
#     # Utwórz nowy plik z nagłówkiem
#     result_row.to_csv(csv_path, index=False)

# print("✅ Wyniki zostały zapisane do pearson_results.csv")


In [79]:
from scipy.stats import spearmanr
import pandas as pd
import os

# Liczenie korelacji i p-value dla entropy
entropy_r_s, entropy_p_s = spearmanr(df_metrics['entropy'], df_metrics['is_error'])

# Liczenie korelacji i p-value dla margin
margin_r_s, margin_p_s = spearmanr(df_metrics['margin'], df_metrics['is_error'])

# Przygotowanie wiersza z wynikami
result_row_spearman = pd.DataFrame([{
    'dataset': dataset_tag,
    'model': model,
    'entropy_corr': entropy_r_s,
    'entropy_p': entropy_p_s,
    'margin_corr': margin_r_s,
    'margin_p': margin_p_s
}])

# Ścieżka do pliku
csv_path_s = 'spearman_results.csv'

# Zapis do pliku
if os.path.exists(csv_path_s):
    result_row_spearman.to_csv(csv_path_s, mode='a', header=False, index=False)
else:
    result_row_spearman.to_csv(csv_path_s, index=False)

print("✅ Wyniki Spearmana + p-value zapisane do spearman_results.csv")


✅ Wyniki Spearmana + p-value zapisane do spearman_results.csv


In [80]:
from scipy.stats import mannwhitneyu
import pandas as pd
import os

# Podział danych na grupy w zależności od wartości zmiennej binarnej 'is_error'
group_0 = df_metrics[df_metrics['is_error'] == 0]
group_1 = df_metrics[df_metrics['is_error'] == 1]

# Test Manna-Whitneya dla zmiennej 'entropy'
entropy_u_stat, entropy_p_value = mannwhitneyu(
    group_0['entropy'], 
    group_1['entropy'], 
    alternative='two-sided'
)

# Test Manna-Whitneya dla zmiennej 'margin'
margin_u_stat, margin_p_value = mannwhitneyu(
    group_0['margin'], 
    group_1['margin'], 
    alternative='two-sided'
)

# Przygotowanie wiersza z wynikami
result_row_mw = pd.DataFrame([{
    'dataset': dataset_tag,
    'model': model,
    'entropy_u_stat': entropy_u_stat,
    'entropy_p_value': entropy_p_value,
    'margin_u_stat': margin_u_stat,
    'margin_p_value': margin_p_value
}])

# Ścieżka do pliku
csv_path_mw = 'mannwhitney_results.csv'

# Zapis do pliku CSV
if os.path.exists(csv_path_mw):
    result_row_mw.to_csv(csv_path_mw, mode='a', header=False, index=False)
else:
    result_row_mw.to_csv(csv_path_mw, index=False)

print("✅ Wyniki testu Manna-Whitneya zapisane do mannwhitney_results.csv")


✅ Wyniki testu Manna-Whitneya zapisane do mannwhitney_results.csv


In [96]:
result_row_mw

,dataset,model,entropy_u_stat,entropy_p_value,margin_u_stat,margin_p_value
0,ag_news,cohere,34924.0,4.841677e-43,139536.0,2.576277e-42


In [81]:
from scipy.stats import pointbiserialr

r, p = pointbiserialr(df_metrics['is_error'], df_metrics['entropy'])
print(f"Point-biserial r = {r:.4f}, p = {p:.4e}")


Point-biserial r = 0.4092, p = 1.2059e-41


In [82]:
r, p = pointbiserialr(df_metrics['is_error'], df_metrics['margin'])
print(f"Point-biserial r = {r:.4f}, p = {p:.4e}")

Point-biserial r = -0.3806, p = 8.1660e-36


In [83]:
r, p = pointbiserialr(df_metrics['is_error'], df_metrics['kl_divergence'])
print(f"Point-biserial r = {r:.4f}, p = {p:.4e}")

Point-biserial r = -0.4092, p = 1.2059e-41


In [84]:
print(df['original_label_map'].apply(type).value_counts())
print(df['output'].apply(type).value_counts())


original_label_map
<class 'str'>    1000
Name: count, dtype: int64
output
<class 'str'>    1000
Name: count, dtype: int64


In [85]:
df

,text,output,logprobs,top_logprobs,original_label,original_label_map,is_error,logprobs_cal,probs,entropy,margin,kl_divergence,normalized_entropy,normalized_margin,normalized_kl_divergence
0,Coke is it: BHP coal to double BHP Billiton pl...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business,2,False,"[-0.003556119976565242, -6.253556251525879, -6...","[0.9964517888610972, 0.001923604224309573, 0.0...",0.038194,0.994528,2.969258,0.018293,0.994694,0.981707
1,BIG HIKE IN ENERGY BILLS The company says the ...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business,2,False,"[-0.009564409963786602, -5.259564399719238, -5...","[0.9904829752455664, 0.005197577689923865, 0.0...",0.090749,0.985285,2.932830,0.044711,0.985449,0.955289
2,Officials Blame Contractors in Tunnel Leak The...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.5...",Business,2,True,"[-0.5680442452430725, -1.5680441856384277, -2....","[0.5666347831743106, 0.2084532998072513, 0.126...",1.642682,0.358181,1.857112,0.824853,0.358241,0.175147
3,Before the Bell: Krispy Kreme Down 11 Pct Shar...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Business,2,False,"[-0.0011151769431307912, -7.251115322113037, -...","[0.9988982266548715, 0.0007093918346481757, 0....",0.014076,0.998189,2.985976,0.006168,0.998355,0.993832
4,Medical Examiner Finds No Injuries on Thanou-S...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.0...",Sports,1,False,"[-0.06411883234977722, -2.8141188621520996, -6...","[0.9379121376099231, 0.0599587151703046, 0.001...",0.351468,0.877953,2.752113,0.175772,0.878099,0.824228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Indian politician sent to prison Top Hindu nat...,0,"{'content': [{'token': '0', 'bytes': [48], 'lo...","[{'token': '0', 'bytes': [48], 'logprob': -0.0...",World,0,False,"[-0.002764572389423847, -6.002764701843262, -8...","[0.9972476572818924, 0.0024719294811621382, 0....",0.029086,0.994776,2.975571,0.013714,0.994941,0.986286
996,Ravens - Bills: Game Notes COMPOSED: With Lewi...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.0...",Sports,1,False,"[-0.0001284993631998077, -9.750128746032715, -...","[0.9998749233985834, 5.828735805473251e-05, 4....",0.002046,0.999817,2.994314,0.000121,0.999983,0.999879
997,Dell touts in-home services Dell offers to ins...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",Sci/Tech,3,True,"[-0.026902785524725914, -3.776902675628662, -5...","[0.9734597554036336, 0.022893581644549816, 0.0...",0.194387,0.950566,2.860994,0.096809,0.950724,0.903191
998,"Lately, it's been fast times World Series anal...",1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.0...",Sports,1,False,"[-0.003462155582383275, -6.003462314605713, -7...","[0.9965683828814522, 0.002470245655437552, 0.0...",0.037434,0.994098,2.969785,0.017910,0.994263,0.982090


In [86]:
incorrect_cases = df['original_label_map'] != df['output']
num_errors = incorrect_cases.sum()   
num_errors

226

In [87]:
# # Wybierz 60 najbardziej niepewnych przypadków (wysoka KL-divergencja)
# uncertain_cases = df_metrics.sort_values(by='kl_divergence', ascending=False)

# # Filtruj przypadki, gdzie predykcja jest błędna
# incorrect_cases = uncertain_cases[uncertain_cases['original_label_map'] != uncertain_cases['output']]



In [88]:
# num_errors = len(incorrect_cases)
# print(f"Liczba błędnych predykcji wśród najbardziej niepewnych przypadków: {num_errors}")


In [89]:
N = num_errors

# 1. Top N najmniej pewnych wg KL (rosnąco)
top_kl = df_metrics.sort_values(by='kl_divergence', ascending=True).head(N)

# 2. Top N najmniej pewnych wg Entropy (malejąco)
top_entropy = df_metrics.sort_values(by='entropy', ascending=False).head(N)

# 3. Filtrujemy tylko błędne przypadki
errors_kl = top_kl[top_kl['original_label_map'] != top_kl['output']]
errors_entropy = top_entropy[top_entropy['original_label_map'] != top_entropy['output']]

# 4. Sprawdzamy, czy wszystkie błędne przypadki z KL są w błędnych z Entropy
kl_indices = set(errors_kl.index)
entropy_indices = set(errors_entropy.index)

is_subset = kl_indices.issubset(entropy_indices)

print(f"Czy wszystkie błędne przypadki z KL są w Entropy? ➡ {is_subset}")
print(f"Liczba błędów w KL: {len(errors_kl)}, w Entropy: {len(errors_entropy)}")


Czy wszystkie błędne przypadki z KL są w Entropy? ➡ True
Liczba błędów w KL: 113, w Entropy: 113


In [90]:
kl_indices

{2,
 8,
 10,
 12,
 14,
 15,
 38,
 53,
 67,
 70,
 86,
 96,
 112,
 128,
 145,
 146,
 148,
 164,
 183,
 190,
 212,
 216,
 217,
 218,
 222,
 227,
 229,
 240,
 253,
 260,
 268,
 280,
 291,
 305,
 311,
 313,
 318,
 339,
 361,
 378,
 380,
 399,
 417,
 421,
 442,
 445,
 457,
 463,
 475,
 476,
 499,
 504,
 508,
 509,
 515,
 522,
 526,
 547,
 561,
 575,
 579,
 583,
 591,
 598,
 599,
 616,
 628,
 637,
 646,
 655,
 659,
 661,
 682,
 684,
 701,
 720,
 736,
 743,
 746,
 747,
 748,
 749,
 752,
 754,
 762,
 764,
 767,
 768,
 770,
 775,
 776,
 786,
 793,
 810,
 813,
 815,
 816,
 829,
 831,
 837,
 854,
 856,
 859,
 876,
 899,
 903,
 905,
 920,
 934,
 942,
 943,
 952,
 961}

In [91]:
entropy_indices

{2,
 8,
 10,
 12,
 14,
 15,
 38,
 53,
 67,
 70,
 86,
 96,
 112,
 128,
 145,
 146,
 148,
 164,
 183,
 190,
 212,
 216,
 217,
 218,
 222,
 227,
 229,
 240,
 253,
 260,
 268,
 280,
 291,
 305,
 311,
 313,
 318,
 339,
 361,
 378,
 380,
 399,
 417,
 421,
 442,
 445,
 457,
 463,
 475,
 476,
 499,
 504,
 508,
 509,
 515,
 522,
 526,
 547,
 561,
 575,
 579,
 583,
 591,
 598,
 599,
 616,
 628,
 637,
 646,
 655,
 659,
 661,
 682,
 684,
 701,
 720,
 736,
 743,
 746,
 747,
 748,
 749,
 752,
 754,
 762,
 764,
 767,
 768,
 770,
 775,
 776,
 786,
 793,
 810,
 813,
 815,
 816,
 829,
 831,
 837,
 854,
 856,
 859,
 876,
 899,
 903,
 905,
 920,
 934,
 942,
 943,
 952,
 961}

In [92]:
from scipy.optimize import minimize

def objective(weights):
    w1, w2, w3 = weights
    df_metrics['combined_score'] = (
        w1 * df_metrics['normalized_entropy'] -
        w2 * df_metrics['normalized_margin'] +
        w3 * (1 - df_metrics['normalized_kl_divergence'])
    )
    top_n = df_metrics.sort_values(by='combined_score', ascending=False).head(N)
    errors = (top_n['original_label_map'] != top_n['output']).sum()
    
    # Dodaj np. penalizację za zbyt niską różnorodność scoringu
    score_std = df_metrics['combined_score'].std()
    
    return -(errors + 0.1 * score_std)  # Minimalizujemy negatywne


res = minimize(objective, x0=[1,1,1], bounds=[(-10,10), (-10,10), (-10,10)], method='L-BFGS-B')

print(f"Optymalne wagi: {res.x}")
print(f"Maks liczba błędów: {-res.fun}")


Optymalne wagi: [10. 10. 10.]
Maks liczba błędów: 113.6621794893308


In [93]:
# 1. Pobierz optymalne wagi
w1, w2, w3 = res.x

# 2. Oblicz combined_score na podstawie najlepszych wag
df_metrics['combined_score'] = (
    w1 * df_metrics['normalized_entropy'] -
    w2 * df_metrics['normalized_margin'] +
    w3 * (1 - df_metrics['normalized_kl_divergence'])
)

# 3. Wybierz top-N najbardziej "niepewnych" przypadków
top_n = df_metrics.sort_values(by='combined_score', ascending=False).head(num_errors)

# 4. Filtruj błędne predykcje
incorrect_cases = top_n[top_n['original_label_map'] != top_n['output']]

# 5. Wyświetl indeksy tych błędnych przypadków
print(incorrect_cases.index.tolist())


[212, 768, 659, 218, 38, 227, 831, 522, 145, 12, 526, 216, 583, 378, 8, 164, 655, 2, 813, 684, 575, 749, 856, 616, 305, 421, 854, 786, 457, 128, 190, 767, 579, 268, 240, 67, 599, 598, 86, 96, 445, 508, 720, 229, 70, 942, 463, 961, 743, 815, 361, 217, 637, 752, 311, 899, 920, 146, 291, 810, 399, 934, 222, 682, 859, 15, 148, 747, 442, 837, 746, 816, 475, 14, 829, 10, 499, 661, 504, 53, 903, 952, 183, 253, 318, 280, 764, 776, 509, 736, 476, 339, 748, 770, 646, 905, 515, 762, 628, 591, 943, 876, 112, 561, 775, 380, 547, 754, 260, 701, 584, 793, 313]


In [94]:
# import numpy as np

# best_score = -np.inf
# best_weights = None

# num_trials = 100000  # Liczba losowych prób

# for _ in range(num_trials):
#     # Losowanie wag z zakresu -10 do 10
#     w2, w3 = np.random.uniform(-10, 10, 2)
    
#     df_metrics['combined_score'] = (
#         w2 * (1-df_metrics['normalized_margin']) +
#         w3 * (df_metrics['normalized_kl_divergence'])
#     )
#     top_n = df_metrics.sort_values(by='combined_score', ascending=False).head(num_errors)
#     errors = (top_n['original_label_map'] != top_n['output']).sum()
    
#     if errors > best_score:
#         best_score = errors
#         best_weights = (w2, w3)

# print(f"Najlepsze wagi po {num_trials} próbach: w1={best_weights[0]:.2f}, w2={best_weights[1]:.2f}")
# print(f"Maksymalna liczba błędów: {best_score}")


In [95]:
# # 1. Oblicz dodatkową miarę: 1 - normalized_margin
# df_metrics['inv_normalized_margin'] = 1 - df_metrics['normalized_margin']

# # 2. Wybierz TOP 50 najmniej pewnych przykładów dla każdej metryki
# top50_entropy = df_metrics.sort_values('normalized_entropy', ascending=False).head(50).index
# top50_margin = df_metrics.sort_values('inv_normalized_margin', ascending=False).head(50).index
# top50_kl = df_metrics.sort_values('normalized_kl_divergence', ascending=True).head(50).index

# # 3. Zamień na zbiory dla łatwego porównania
# set_entropy = set(top50_entropy)
# set_margin = set(top50_margin)
# set_kl = set(top50_kl)

# # 4. Analiza przecięć i unikalnych przypadków
# common_all = set_entropy & set_margin & set_kl
# common_entropy_margin = set_entropy & set_margin
# common_entropy_kl = set_entropy & set_kl
# common_margin_kl = set_margin & set_kl

# unique_entropy = set_entropy - (set_margin | set_kl)
# unique_margin = set_margin - (set_entropy | set_kl)
# unique_kl = set_kl - (set_entropy | set_margin)

# # 5. Wyniki
# print(f"Wspólne w 3 metrykach: {len(common_all)}")
# print(f"Wspólne entropy & margin: {len(common_entropy_margin)}")
# print(f"Wspólne entropy & kl: {len(common_entropy_kl)}")
# print(f"Wspólne margin & kl: {len(common_margin_kl)}")
# print(f"Unikalne dla entropy: {len(unique_entropy)}")
# print(f"Unikalne dla margin: {len(unique_margin)}")
# print(f"Unikalne dla kl: {len(unique_kl)}")

# # 6. (Opcjonalnie) Zobacz które indeksy się powtarzają:
# print("Indeksy wspólne dla wszystkich 3 metryk:", common_all)
